In [2]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

In [16]:
# Создаём сэмпл
n_samples = 1000

month = np.random.choice(12, n_samples)
job_category = np.random.randint(3, 8, n_samples)
day_per_month = np.random.randint(15, 20, n_samples)
number_of_parts_per_day = np.random.choice(20, n_samples)
order_number = np.random.choice(1000000, n_samples)
cost_category = 100
bonus = 1000
salary = day_per_month * number_of_parts_per_day * job_category * cost_category + bonus

data = pd.DataFrame({'month': month, 'job_category': job_category, 'day_per_month': day_per_month, 
                     'number_of_parts_per_day': number_of_parts_per_day, 'order_number': order_number,
                     'cost_category': cost_category, 'salary': salary})
data.head(5)

,month,job_category,day_per_month,number_of_parts_per_day,order_number,cost_category,salary
0,5,6,19,16,563239,100,183400
1,0,7,15,13,215356,100,137500
2,7,3,19,1,202806,100,6700
3,0,7,17,17,41931,100,203300
4,7,6,19,0,77298,100,1000


**Зарплата рабочего зависит от количества рабочих дней месяца, количества деталей обработанных за день, категории работы к которой относится деталь (чем выше категория тем дороже работа), стоимости работы и премии. \
Допущения - в один день одна категория работ, премия всегда равна 1000 руб. \
Номер заказа и месяц в расчете запрлаты не участвуют**

In [19]:
from sklearn.metrics import mean_absolute_error

X = data[['month', 'job_category', 'day_per_month', 'number_of_parts_per_day', 'order_number', 'cost_category']]
y = data['salary']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['month', 'job_category', 'day_per_month', 'number_of_parts_per_day', 'order_number', 'cost_category']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 7.84629814e+00  1.56095705e+04  4.58075347e+03  8.60401389e+03
 -8.54941593e-04  0.00000000e+00]
Bias: -156240.9021936875
Error: 10875.214624981467


**На первом шаге пробуем спрогнозировать зарплату исходя из всех признаков. \
Результат явно не оптимальный, смещение 156 240, хотя премия всегда одинакова и равна 1000 руб. \
Абсолютная ошибка более 10000 тысяч.**\
*Следующим шагом применим при расчете только влияющие на расчет признаки*

In [20]:
from sklearn.metrics import mean_absolute_error

X = data[['job_category', 'day_per_month', 'number_of_parts_per_day', 'cost_category']]
y = data['salary']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['job_category', 'day_per_month', 'number_of_parts_per_day', 'cost_category']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [15610.42813812  4585.07467482  8606.65526549     0.        ]
Bias: -156726.34836365533
Error: 10885.315957321356


**Результат снова неутешительный. Смещение и ошибка на том же уровне** \
*Создадим новый признак - выработка в месяц*

In [21]:
# Создаем новый признак
data['production'] = data['job_category'] * data['day_per_month'] * data['number_of_parts_per_day']
data.head(5)

,month,job_category,day_per_month,number_of_parts_per_day,order_number,cost_category,salary,production
0,5,6,19,16,563239,100,183400,1824
1,0,7,15,13,215356,100,137500,1365
2,7,3,19,1,202806,100,6700,57
3,0,7,17,17,41931,100,203300,2023
4,7,6,19,0,77298,100,1000,0


In [25]:
from sklearn.metrics import mean_absolute_error

X = data[['production', 'cost_category']]
y = data['salary']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['production', 'cost_category']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [100.   0.]
Bias: 999.9999999999854
Error: 1.3305907486937941e-11


**При использовании агрегирующего признака модель строит точное предсказание, т.к. модель линейна и предсказывает веса каждого признака. Однако здесь есть признак категория работы с весом = 0, это потому что признак является константой.** \
*Уберем этот признак*

In [40]:
from sklearn.metrics import mean_absolute_error

X = data[['production']]
y = data['salary']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print(f'Bias: {np.round(reg.intercept_)}')

pred_values = reg.predict(data[['production']])
print(f'Bias: {np.round(mean_absolute_error(pred_values, y))}')

Weights: [100.]
Bias: 1000.0
Bias: 0.0


**Теперь видим стопроцентную точность предсказания**